In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [11]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  
    transforms.RandomRotation(10),  
    transforms.RandomCrop(32, padding=4),  
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [12]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define CNN layers
        self.conv1 = nn.Conv2d(3,32,kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32,64,kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128,kernel_size=3,  padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2,2)

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 2 * 2, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)  
        
        self.dropout1 = nn.Dropout(0.5)  
        self.dropout2 = nn.Dropout(0.5)      

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #  feature map + ReLU + Maxpool
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))

        x = x.view(-1, 256 * 2 *2) 

        x = F.relu(self.fc1(x))
        x = self.dropout1(x)  

        x = F.relu(self.fc2(x))
        x = self.dropout2(x)   

        x = self.fc3(x) 
        return x
    
net = Net()

In [14]:
criterion = nn.CrossEntropyLoss() 

# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) 
optimizer = optim.Adam(net.parameters(),lr=0.001)

net = net.to(device)
criterion = criterion.to(device)


In [ ]:
for epoch in range(50): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()


    epoch_loss = running_loss / len(trainloader)  # Average loss per epoch
    print(f'Epoch [{epoch + 1}/30], Loss: {epoch_loss:.4f}')
print('Finished Training')

torch.save(net.state_dict(), 'cifar_net.pth')

Epoch [1/30], Loss: 1.7365
Epoch [2/30], Loss: 1.3631
Epoch [3/30], Loss: 1.2198
Epoch [4/30], Loss: 1.1388
Epoch [5/30], Loss: 1.0825
Epoch [6/30], Loss: 1.0328
Epoch [7/30], Loss: 1.0051
Epoch [8/30], Loss: 0.9719
Epoch [9/30], Loss: 0.9491
Epoch [10/30], Loss: 0.9237
Epoch [11/30], Loss: 0.9069
Epoch [12/30], Loss: 0.8985
Epoch [13/30], Loss: 0.8851
Epoch [14/30], Loss: 0.8672
Epoch [15/30], Loss: 0.8615
Epoch [16/30], Loss: 0.8529
Epoch [17/30], Loss: 0.8485
Epoch [18/30], Loss: 0.8411
Epoch [19/30], Loss: 0.8235
Epoch [20/30], Loss: 0.8258
Epoch [21/30], Loss: 0.8057
Epoch [22/30], Loss: 0.7976
Epoch [23/30], Loss: 0.8079
Epoch [24/30], Loss: 0.7993
Epoch [25/30], Loss: 0.7994
Epoch [26/30], Loss: 0.8026
Epoch [27/30], Loss: 0.7980
Epoch [28/30], Loss: 0.7890
Epoch [29/30], Loss: 0.7861
Epoch [30/30], Loss: 0.7840
Epoch [31/30], Loss: 0.7826
Epoch [32/30], Loss: 0.7757
Epoch [33/30], Loss: 0.7713
Epoch [34/30], Loss: 0.7755
Epoch [35/30], Loss: 0.7721
Epoch [36/30], Loss: 0.7642
E

In [16]:
correct = 0
total = 0

net.eval()

with torch.no_grad(): 
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Training Accuracy: {100 * correct / total:.2f}%")

Training Accuracy: 78.25%


In [17]:
net.load_state_dict(torch.load('cifar_net.pth'))

correct = 0
total = 0
with torch.no_grad(): 
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')
  

C:\Users\sayed\AppData\Local\Temp\ipykernel_17924\3478249005.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('cifar_net.pth'))


Accuracy: 78.25%


In [18]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

correct = [0] * 10  
total = [0] * 10
total_correct = 0
total_samples = 0

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs, 1)

        # Update correct and total counts for each class
        for i in range(len(labels)):
            label = labels[i].item()
            total[label] += 1
            total_samples += 1
            if predicted[i] == label:
                correct[label] += 1
                total_correct += 1

# Calculate the accuracy for each class
class_accuracies = []
for i in range(10):
    accuracy = (correct[i] / total[i]) * 100
    class_accuracies.append((classes[i], accuracy))

class_accuracies.sort(key=lambda x: x[1])

for class_name, accuracy in class_accuracies:
    print(f"Accuracy for class {class_name}: {accuracy:.2f}%")

overall_accuracy = (total_correct / total_samples) * 100
print(f"Overall Accuracy: {overall_accuracy:.2f}%")


Accuracy for class cat: 64.40%
Accuracy for class bird: 65.00%
Accuracy for class dog: 73.90%
Accuracy for class deer: 74.80%
Accuracy for class horse: 80.20%
Accuracy for class plane: 81.30%
Accuracy for class car: 83.30%
Accuracy for class frog: 83.80%
Accuracy for class truck: 87.30%
Accuracy for class ship: 88.50%
Overall Accuracy: 78.25%
